In [2]:
import cv2

video_path = "data/session5_center/video.avi"
json_gtruth_path = 'data/session5_center_gtruth/system_dubska_optimal_calib.json'

In [7]:
import pickle

with open('data/session5_center/gt_data.pkl', 'rb') as pkl_file:
    speed_data = pickle.load(pkl_file, encoding='latin')

In [8]:
speed_data.keys()

dict_keys(['distanceMeasurement', 'measurementLines', 'cars', 'fps', 'invalidLanes', 'laneDivLines'])

In [16]:
import pandas as pd

speed_df = pd.read_pickle('data/session5_center/gt_data.pkl')

speed_df['distanceMeasurement']


[{'p2': array([705.8898615 , 163.39016805,   1.        ]),
  'distance': 6.98,
  'toVP1': True,
  'p1': array([774.0498615 ,  85.37706371,   1.        ])},
 {'p2': array([601.59290859, 283.80046537,   1.        ]),
  'distance': 7.013,
  'toVP1': True,
  'p1': array([705.8898615 , 163.39016805,   1.        ])},
 {'p2': array([423.06269252, 491.47745152,   1.        ]),
  'distance': 7.014,
  'toVP1': True,
  'p1': array([601.59290859, 283.80046537,   1.        ])},
 {'p2': array([ 78.91359557, 909.30136288,   1.        ]),
  'distance': 6.985,
  'toVP1': True,
  'p1': array([423.06269252, 491.47745152,   1.        ])},
 {'p2': array([ 78.91359557, 909.30136288,   1.        ]),
  'distance': 27.998,
  'toVP1': True,
  'p1': array([774.0498615 ,  85.37706371,   1.        ])},
 {'p2': array([1.27372481e+03, 8.58372964e+01, 1.00000000e+00]),
  'distance': 27.998,
  'toVP1': True,
  'p1': array([1.59889463e+03, 9.67573629e+02, 1.00000000e+00])},
 {'p2': array([911.29688643,  85.05014958,   

In [9]:
speed_data['distanceMeasurement']

[{'p2': array([705.8898615 , 163.39016805,   1.        ]),
  'distance': 6.98,
  'toVP1': True,
  'p1': array([774.0498615 ,  85.37706371,   1.        ])},
 {'p2': array([601.59290859, 283.80046537,   1.        ]),
  'distance': 7.013,
  'toVP1': True,
  'p1': array([705.8898615 , 163.39016805,   1.        ])},
 {'p2': array([423.06269252, 491.47745152,   1.        ]),
  'distance': 7.014,
  'toVP1': True,
  'p1': array([601.59290859, 283.80046537,   1.        ])},
 {'p2': array([ 78.91359557, 909.30136288,   1.        ]),
  'distance': 6.985,
  'toVP1': True,
  'p1': array([423.06269252, 491.47745152,   1.        ])},
 {'p2': array([ 78.91359557, 909.30136288,   1.        ]),
  'distance': 27.998,
  'toVP1': True,
  'p1': array([774.0498615 ,  85.37706371,   1.        ])},
 {'p2': array([1.27372481e+03, 8.58372964e+01, 1.00000000e+00]),
  'distance': 27.998,
  'toVP1': True,
  'p1': array([1.59889463e+03, 9.67573629e+02, 1.00000000e+00])},
 {'p2': array([911.29688643,  85.05014958,   

In [21]:
import json

with open(json_gtruth_path, 'r') as json_file:
    position_data = json.load(json_file)

cars_data = position_data['cars']

In [55]:
cap = cv2.VideoCapture(video_path)


frame_number = 0
i = 0
j = 0


while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_list = cars_data[i]['frames']
    car_id = cars_data[i]['id']
    
    if frame_list[j] == frame_number:
        pos_x = int(cars_data[i]['posX'][j])
        pos_y = int(cars_data[i]['posY'][j])
        
        img = cv2.circle(frame, (pos_x, pos_y), 20, (0,0,255), -1)
        img = cv2.putText(img, f"Car number {car_id}", (pos_x, pos_y), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0))
        j+=1
        
        if j == len(frame_list):
            i += 1
            j = 0
    
    else:
        img = frame
    

    
    cv2.imshow('frame', img)
        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
    
    frame_number += 1
    

    
cap.release()
cv2.destroyAllWindows()

In [50]:
cars_data[0]['id']

0